<a href="https://colab.research.google.com/github/Andres8bit/parallel-computing/blob/main/histogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-6ac3y5se
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-6ac3y5se
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=5b40e5615f341a3dd057a84398ab258304dd8062071e54daaaa1ddb4f53bc52d
  Stored in directory: /tmp/pip-ephem-wheel-cache-g8ny0qd1/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [ ]:
%%cu
#include<iostream>


__global__ void hist_blocking_kernel(unsigned char* buffer, long size,unsigned int* histo);
void print_array(unsigned int *array, int width);

int main(){
    unsigned int host_histo[] = {0,0,0,0,0,0,0};
    unsigned char host_buffer[] = {'p','r','o','g','r','a','m','m','i','n','g',' ','m',
                          'a','s','s','i','v','e','l','y',' ','p','a'};
    
    long size = 24;
    int buff_size = size * sizeof(char);
    int hist_size = 7 * sizeof(int);

    unsigned char *device_buffer;
    unsigned int *device_histo;

    cudaMalloc((void **) &device_buffer,buff_size);
    cudaMalloc((void **) &device_histo,hist_size);

    cudaMemcpy(device_buffer,host_buffer,buff_size,cudaMemcpyHostToDevice);
    cudaMemcpy(device_histo,host_histo,hist_size,cudaMemcpyHostToDevice);

    dim3 dim_block(6);
    dim3 dim_grid(4);

    hist_blocking_kernel<<<dim_grid,dim_block>>>(device_buffer,size,device_histo); 

    cudaMemcpy(host_histo,device_histo,hist_size,cudaMemcpyDeviceToHost);

    print_array(host_histo,7);

    return 0;
}

void print_array(unsigned int *array, int width){
        for(int i = 0; i < width; i++)
        printf("%d\t", (int)array[i]);
    
  printf("\n");
}
__global__ void hist_blocking_kernel(unsigned char* buffer, long size,unsigned int* histo){
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int section_size = (size - 1) / (blockDim.x * gridDim.x) + 1;
    int start = i * section_size;
    int alphabet_position =  0;
    
    for (int k = 0; k < section_size; k++){
        if ( start + k < size){
          alphabet_position = buffer[start + k] -'a';
          if (alphabet_position >= 0 && alphabet_position < 26)
              atomicAdd(&(histo[alphabet_position/4]), 1);
        }
    }
}

3	3	3	7	4	1	1	



In [ ]:
%%cu
#include<iostream>

__global__ void hist_interleaved_kernel(unsigned char* buffer,long size, 
                                        unsigned int* histo);
                                  
void print_array(unsigned int *array, int width);

int main(){
        unsigned int host_histo[] = {0,0,0,0,0,0,0};
    unsigned char host_buffer[] = {'p','r','o','g','r','a','m','m','i','n','g',' ','m',
                          'a','s','s','i','v','e','l','y',' ','p','a'};
    
    long size = 24;
    int buff_size = size * sizeof(char);
    int hist_size = 7 * sizeof(int);

    unsigned char *device_buffer;
    unsigned int *device_histo;

    cudaMalloc((void **) &device_buffer,buff_size);
    cudaMalloc((void **) &device_histo,hist_size);

    cudaMemcpy(device_buffer,host_buffer,buff_size,cudaMemcpyHostToDevice);
    cudaMemcpy(device_histo,host_histo,hist_size,cudaMemcpyHostToDevice);

    dim3 dim_block(6);
    dim3 dim_grid(4);

  hist_interleaved_kernel<<<dim_grid,dim_block>>>(device_buffer,size,device_histo); 

    cudaMemcpy(host_histo,device_histo,hist_size,cudaMemcpyDeviceToHost);

    print_array(host_histo,7);

    return 0;
}

void print_array(unsigned int *array, int width){
        for(int i = 0; i < width; i++)
        printf("%d\t", (int)array[i]);
    
  printf("\n");

}

__global__ void hist_interleaved_kernel(unsigned char* buffer,long size, 
                                        unsigned int* histo)
{
  unsigned int thread_id = threadIdx.x + blockIdx.x * blockDim.x;

   int alphabet_pos = 0;
  for ( unsigned int i = thread_id; i < size; i += blockDim.x * gridDim.x){
      alphabet_pos = buffer[i] - 'a';

      if (alphabet_pos >= 0 && alphabet_pos < 26)
          atomicAdd(&(histo[alphabet_pos/4]),1);
  }                                             
}

3	3	3	7	4	1	1	



In [ ]:
%%cu
#include<iostream>

__global__ void hist_privatized_kernel(unsigned char* buffer,unsigned int* histo,
                              unsigned int num_elements, unsigned int num_bins);

void print_array(unsigned int *array, int width);

int main(){
        unsigned int host_histo[] = {0,0,0,0,0,0,0};
    unsigned char host_buffer[] = {'p','r','o','g','r','a','m','m','i','n','g',' ','m',
                          'a','s','s','i','v','e','l','y',' ','p','a'};
    
    long size = 24;
    int buff_size = size * sizeof(char);
    int hist_size = 7 * sizeof(int);

    unsigned char *device_buffer;
    unsigned int *device_histo;

    cudaMalloc((void **) &device_buffer,buff_size);
    cudaMalloc((void **) &device_histo,hist_size);

    cudaMemcpy(device_buffer,host_buffer,buff_size,cudaMemcpyHostToDevice);
    cudaMemcpy(device_histo,host_histo,hist_size,cudaMemcpyHostToDevice);

    dim3 dim_block(6);
    dim3 dim_grid(4);

  hist_privatized_kernel<<<dim_grid,dim_block>>>(device_buffer,device_histo,24,7); 

    cudaMemcpy(host_histo,device_histo,hist_size,cudaMemcpyDeviceToHost);

    print_array(host_histo,7);

    return 0;
}

void print_array(unsigned int *array, int width){
        for(int i = 0; i < width; i++)
        printf("%d\t", (int)array[i]);
    
  printf("\n");

}
__global__ void hist_privatized_kernel(unsigned char* buffer,unsigned int* histo,
                              unsigned int num_elements, unsigned int num_bins)
{
    unsigned int thread_id = blockIdx.x * blockDim.x + threadIdx.x;
    extern __shared__ unsigned int histo_s[7];
 
    for (unsigned int binId = threadIdx.x; binId < num_bins; binId += blockDim.x)
        histo_s[binId] = 0;
 
    __syncthreads();
 
    int alphabet_pos = 0;
    for (unsigned int i = thread_id; i < num_elements; i += blockDim.x * gridDim.x)
    {
      alphabet_pos = buffer[i] - 'a';
      if (alphabet_pos >= 0 && alphabet_pos < 26)
        atomicAdd(&(histo_s[alphabet_pos/4]),1);   
    }
 
   __syncthreads();
 
   for (unsigned int binId = threadIdx.x; binId < num_bins; binId += blockDim.x)
       atomicAdd(&(histo[binId]),histo_s[binId]);
}

3	3	3	7	4	1	1	



In [ ]:
%%cu
#include <iostream>

__global__ void hist_aggr_pvt_kernel(unsigned char* buffer, unsigned int* hist 
                            ,unsigned int num_elements, unsigned int num_bins);
void print_array(unsigned int *array, int width);

int main(){
        unsigned int host_histo[] = {0,0,0,0,0,0,0};
    unsigned char host_buffer[] = {'p','r','o','g','r','a','m','m','i','n','g',' ','m',
                          'a','s','s','i','v','e','l','y',' ','p','a'};
    
    long size = 24;
    int buff_size = size * sizeof(char);
    int hist_size = 7 * sizeof(int);

    unsigned char *device_buffer;
    unsigned int *device_histo;

    cudaMalloc((void **) &device_buffer,buff_size);
    cudaMalloc((void **) &device_histo,hist_size);

    cudaMemcpy(device_buffer,host_buffer,buff_size,cudaMemcpyHostToDevice);
    cudaMemcpy(device_histo,host_histo,hist_size,cudaMemcpyHostToDevice);

    dim3 dim_block(6);
    dim3 dim_grid(4);

  hist_aggr_pvt_kernel<<<dim_grid,dim_block>>>(device_buffer,device_histo,24,7); 

    cudaMemcpy(host_histo,device_histo,hist_size,cudaMemcpyDeviceToHost);

    print_array(host_histo,7);

    return 0;
}

void print_array(unsigned int *array, int width){
        for(int i = 0; i < width; i++)
        printf("%d\t", (int)array[i]);
    
  printf("\n");

}
__global__ void hist_aggr_pvt_kernel(unsigned char* buffer, unsigned int* hist 
                            ,unsigned int num_elements, unsigned int num_bins)
{
  unsigned int thread_id = blockIdx.x * blockDim.x + threadIdx.x;
  extern __shared__ unsigned int histo_s[7];
 
  for (unsigned int binId = threadIdx.x; binId < num_bins; binId += blockDim.x)
      histo_s[binId] = 0;
 
 __syncthreads();

 unsigned int prev = -1;
 unsigned int accumulator = 0;
 unsigned int curr = 0;
  int pos = 0;

 for (unsigned int i = thread_id; i < num_elements; i += blockDim.x * gridDim.x){
     pos = buffer[i] - 'a';
     if ( pos >= 0 && pos < 26){
        curr = pos/4;
        if (curr != prev){           
            if (accumulator > 0)
                 atomicAdd(&(histo_s[curr]),accumulator);
                 accumulator = 1;
                 prev = curr;
            }else
                accumulator++;
        }     
   }
  __syncthreads();
  for (unsigned int binId = threadIdx.x; binId < num_bins; binId += blockDim.x)
        atomicAdd(&(hist[binId]),histo_s[binId]);
}

0	0	0	0	0	0	0	

